<a href="https://colab.research.google.com/github/nianlonggu/WhisperSeg/blob/master/docs/WhisperSeg_Training_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Pipeline of WhisperSeg

## Clone GitHub repository and install environment

In [2]:
!git clone https://github.com/nianlonggu/WhisperSeg.git
!cd WhisperSeg; pip install -r requirements.txt --quiet

fatal: destination path 'WhisperSeg' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
os.chdir("WhisperSeg")

## Dataset Processing
Here we download the Mouse dataset for demonstration purpose. For the complete datasets and more details, please refer our GitHub repository:
https://github.com/nianlonggu/WhisperSeg/blob/master/docs/DatasetProcessing.md#dataset-processing

In [7]:
from huggingface_hub import snapshot_download
os.makedirs("data/mouse", exist_ok=True)
snapshot_download('nccratliri/vad-mouse', local_dir = "data/mouse", repo_type="dataset" )

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

'/content/WhisperSeg/data/mouse'

## Model Training
We train a WhisperSeg-base model one th Mouse dataset.

For more details on training WhisperSeg, please refer to our GitHub repository: https://github.com/nianlonggu/WhisperSeg/blob/master/docs/ModelTraining.md#model-training

First we need to login wandb to log the training process. Please refer to https://wandb.ai/ for more information.

In [9]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Start training

In [10]:
!python train.py --initial_model_path openai/whisper-base --train_dataset_folder data/mouse/train/  --model_folder model/whisperseg-base-mouse --gpu_list 0

2023-09-11 21:43:33.789450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: nianlonggu (nccratliri). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/WhisperSeg/wandb/run-20230911_214334-kzwhkozm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run different-feather-301
wandb: ⭐️ View project at https://wandb.ai/nccratliri/whisperseg-multi-species
wandb: 🚀 View run at https://wandb.ai/nccratliri/whisperseg-multi-species/runs/kzwhkozm
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:5

## Evaluation
The trained model checkpoint is stored at "model/whisperseg-base-mouse/final_checkpoint" and "model/whisperseg-base-mouse/final_checkpoint_ct2" (CTranslate2 version). We evaluate the performance of the trained model on Mouse's test set.

For more detailed information of evaluation, please refer to our GitHub repository: https://github.com/nianlonggu/WhisperSeg/blob/master/docs/Evaluation.md#evaluation

### Evaluate the model using the frame-wise F1 score ($F1_\text{frame}$) and segment-wise F1 score ($F1_\text{seg}$)

In [11]:
from model import WhisperSegmenterFast, WhisperSegmenter
import librosa
import numpy as np
from tqdm import tqdm
from copy import deepcopy
from train import evaluate
from datautils import get_audio_and_label_paths
import os
from audio_utils import SpecViewer
import subprocess
from glob import glob
import json

In [12]:
def evaluate_dataset( dataset_folder, model_path, num_trials, consolidation_method = "clustering",
                      max_length = 448, num_beams = 4, batch_size = 8 ):
    audio_list, label_list = [], []
    audio_paths, label_paths = get_audio_and_label_paths(dataset_folder)
    for audio_path, label_path in zip(audio_paths, label_paths):
        label = json.load( open( label_path ) )
        audio, _ = librosa.load( audio_path, sr = label["sr"] )
        audio_list.append(audio)
        label_list.append(label)


    segmenter = WhisperSegmenterFast(  model_path = model_path,  device = "cuda")
    res = evaluate( audio_list, label_list, segmenter, batch_size, max_length, num_trials, consolidation_method, num_beams, target_cluster = None )

    all_res = {
        "segment_wise_scores": {"N-true-positive": res["segment_wise"][0],
                                "N-positive-in-prediction": res["segment_wise"][1],
                                "N-positive-in-ground-truth": res["segment_wise"][2],
                                "precision": res["segment_wise"][3],
                                "recall": res["segment_wise"][4],
                                "F1": res["segment_wise"][5]
                                },
        "frame_wise_scores": {"N-true-positive": res["frame_wise"][0],
                                "N-positive-in-prediction": res["frame_wise"][1],
                                "N-positive-in-ground-truth": res["frame_wise"][2],
                                "precision": res["frame_wise"][3],
                                "recall": res["frame_wise"][4],
                                "F1": res["frame_wise"][5]
                                }
    }
    return all_res

In [13]:
evaluate_dataset( "data/mouse/test/", "model/whisperseg-base-mouse/final_checkpoint_ct2", num_trials =3 )

100%|██████████| 3/3 [01:00<00:00, 20.17s/it]


{'segment_wise_scores': {'N-true-positive': 208,
  'N-positive-in-prediction': 222,
  'N-positive-in-ground-truth': 229,
  'precision': 0.9369369369369369,
  'recall': 0.9082969432314411,
  'F1': 0.9223946784922394},
 'frame_wise_scores': {'N-true-positive': 10436,
  'N-positive-in-prediction': 10803,
  'N-positive-in-ground-truth': 10852,
  'precision': 0.9660279551976303,
  'recall': 0.9616660523405823,
  'F1': 0.9638420688062803}}

### Evaluate the model by visualization

In [14]:
from model import WhisperSegmenterFast
from audio_utils import SpecViewer
segmenter = WhisperSegmenterFast( "model/whisperseg-base-mouse/final_checkpoint_ct2", device="cuda" )
spec_viewer = SpecViewer()

In [15]:
sr = 300000
min_frequency = 35000
spec_time_step = 0.0005
min_segment_length = 0.01
eps = 0.02
num_trials = 3

audio_file = "data/example_subset/Mouse/test/mouse_Rfem_Afem01_0.wav"
label_file = audio_file[:-4] + ".json"
audio, _ = librosa.load( audio_file, sr = sr )
label = json.load( open(label_file) )

prediction = segmenter.segment(  audio, sr = sr, min_frequency = min_frequency, spec_time_step = spec_time_step,
                       min_segment_length = min_segment_length, eps = eps,num_trials = num_trials )
spec_viewer.visualize( audio = audio, sr = sr, min_frequency= min_frequency, prediction = prediction, label=label )

interactive(children=(FloatSlider(value=6.5, description='offset', max=13.02541, step=0.25), Output()), _dom_c…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

For more details please refer to our paper:
> [**Positive Transfer of the Whisper Speech Transformer to Human and Animal Voice Activity Detection**]()
>
> Nianlong Gu, Kanghwi Lee, Maris Basha, Sumit Kumar Ram, Guanghao You, Richard H. R. Hahnloser <br>
> University of Zurich and ETH Zurich

GitHub: https://github.com/nianlonggu/WhisperSeg

## Contact:
nianlong.gu@uzh.ch